In [1]:
import sys
from kaitoupao import *

storage dir: /Users/minkexiu/Downloads/GitHub/Tianchi_NLPNewsClassification
code dir: /Users/minkexiu/Documents/GitHub/Tianchi_NLPNewsClassification 

07 22 58
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
本卦上：7 本卦下：6 变爻：4


,山水蒙,地雷复,火水未济
上卦,☶艮土,☷坤土,☲离火
下卦,☵坎水,☳震木,☵坎水


01 10 12 亥时
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
本卦上：1 本卦下：2 变爻：6


,天泽履,风火家人,泽泽兑
上卦,☰乾金,☴巽木,☱兑金
下卦,☱兑金,☲离火,☱兑金


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import f1_score

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
device = torch.device("mps")

# 加载训练集和测试集，将全量字符列表给它弄出来

In [5]:
data_train = pd.read_csv(create_originalData_path("train_set.csv"), sep="\t")#.sample(1000) , nrows=1000
data_test = pd.read_csv(create_originalData_path("test_a.csv"), sep="\t")#.sample(1000)

In [6]:
data_train.shape, data_test.shape

((200000, 2), (50000, 1))

In [7]:
type_of_class = data_train.label.nunique()

In [8]:
train_data, valid_data = train_test_split(data_train, test_size=0.3, random_state=42)

In [9]:
train_labels = torch.tensor(train_data.label.to_list(), dtype=torch.long)
valid_labels = torch.tensor(valid_data.label.to_list(), dtype=torch.long)
test_labels = torch.tensor([-1 for x in range(data_test.shape[0])], dtype=torch.long) ## test_labels的label是假的。

## 使用TF-IDF提取特征

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_train_features = tfidf_vectorizer.fit_transform(train_data['text'])
tfidf_valid_features = tfidf_vectorizer.transform(valid_data['text'])
tfidf_test_features = tfidf_vectorizer.transform(data_test['text'])

In [11]:
train_features = torch.tensor(tfidf_train_features.toarray(), dtype=torch.float32)
valid_features = torch.tensor(tfidf_valid_features.toarray(), dtype=torch.float32)
test_features = torch.tensor(tfidf_test_features.toarray(), dtype=torch.float32)

In [12]:
sc_input_dim = train_features.shape[1]
sc_input_dim

6695

## 创建适合于语言序列的数据

In [13]:
# 下载数据并进行预处理
vocab_size = 8000  # 只考虑前 20k 词汇
maxlen = 800  # 只考虑每条评论的前 200 个词

In [14]:
def preprocess_seq_str_2_int(seq, len_lim = maxlen):
    rst = [int(wd) for idx, wd in enumerate(seq.strip().split()) if idx < len_lim]
    return rst

In [15]:
x_train = [torch.tensor(preprocess_seq_str_2_int(seq), dtype=torch.long) for seq in train_data.text]
x_valid = [torch.tensor(preprocess_seq_str_2_int(seq), dtype=torch.long) for seq in valid_data.text]
x_test = [torch.tensor(preprocess_seq_str_2_int(seq), dtype=torch.long) for seq in data_test.text]

In [16]:
x_train = pad_sequence(x_train, batch_first=True, padding_value=0)
x_valid = pad_sequence(x_valid, batch_first=True, padding_value=0)
x_test = pad_sequence(x_test, batch_first=True, padding_value=0)

## 准备数据集

In [17]:
batchsize = 32

In [18]:
class MyData(Dataset):
    def __init__(
        self, 
        ori_data, tfidf_feats, label,
    ):
        self.ori_data = ori_data
        self.tfidf_feats = tfidf_feats
        self.label = label
 
    def __len__(self):
        return len(self.ori_data)
 
    def __getitem__(self, idx):
        tuple_ = (
            self.ori_data[idx], 
            self.tfidf_feats[idx], 
            self.label[idx]
        )
        return tuple_

In [19]:
train_loader = DataLoader(MyData(x_train, train_features, train_labels), batch_size=batchsize, shuffle=True,)
val_loader = DataLoader(MyData(x_valid, valid_features, valid_labels), batch_size=batchsize, shuffle=True,)
test_loader = DataLoader(MyData(x_test, test_features, test_labels), batch_size=batchsize, shuffle=False,) ## 这个不要shuffle，否则传到oj上面去就GG了。

# 定义网络结构

并行使用CNN+BiLSTM+Transformer：
* CNN捕捉局部n-gram特征（kernel_size=3,5,7）
* BiLSTM捕获长距离时序依赖
* Transformer处理全局关系

In [20]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.att = nn.MultiheadAttention(embed_dim, num_heads)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
        self.layernorm1 = nn.LayerNorm(embed_dim)
        self.layernorm2 = nn.LayerNorm(embed_dim)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.att(x, x, x)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(nn.Module):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.pos_emb = nn.Embedding(maxlen, embed_dim)

    def forward(self, x):
        maxlen = x.size(1)
        positions = torch.arange(0, maxlen, device=x.device).unsqueeze(0).expand(x.size(0), x.size(1)) # torch.arange(0, maxlen, device=x.device).unsqueeze(0).expand_as(x)
        # print(positions.shape)
        return x + self.pos_emb(positions)

class TransformerModel(nn.Module):
    def __init__(self, maxlen, vocab_size, embed_dim, num_heads, ff_dim):
        super(TransformerModel, self).__init__()
        self.embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        
    def forward(self, x):
        x = self.embedding_layer(x).transpose(0, 1)  # Transformer expects (seq_len, batch_size, embed_dim)
        x = self.transformer_block(x)
        x = x.transpose(0, 1)
        x = self.global_avg_pool(x.permute(0, 2, 1)).squeeze(-1)
        return x

In [21]:
## TextCNN：捕捉短时间的关系

# https://blog.51cto.com/u_15764210/6844118
class GlobalMaxPool1d(nn.Module):
    def __init__(self):
        super(GlobalMaxPool1d, self).__init__()
    def forward(self, x):
        return F.max_pool1d(x, kernel_size=x.shape[2]) # shape: (batch_size, channel, 1)
        
class TextCNN(nn.Module):
    def __init__(
        self, 
        embedding_dim=128, 
        kernel_sizes=[3, 4, 5, 6], num_channels=[256, 256, 256, 256], 
    ):
        '''
        ：param num_classes：输出维度（类别数num_Classes）
        ：param num_embeddings: size of the dictionary of embeddings，词典的大小（vocab_size），当num_embeddings<O，模型会去除embedding层
        ：param embedding_dim: the size of each embedding vector，词向量特征长度
        ：param kernel_sizes: CNN层卷积核大小
        ：param num_channels: CNN层卷积核通道数
        : return:
        '''
        assert len(kernel_sizes) == len(num_channels), "len(kernel_sizes) should be equal to len(num_channels)"
        super(TextCNN, self).__init__()
        # self.num_classes = num_classes
    
        # 卷积层
        self.cnn_layers = nn.ModuleList() # 创建多个一维卷积层
        for c, k in zip(num_channels, kernel_sizes):
            cnn = nn.Sequential(
                nn.Conv1d(
                    in_channels=embedding_dim,
                    out_channels=c, 
                    kernel_size=k
                ),
                nn.BatchNorm1d(c),
                nn.ReLU(inplace=True),
            )
            self.cnn_layers.append(cnn)
        # 最大池化层
        self.pool = GlobalMaxPool1d()
            
    def forward(self, input_):
        '''
        :param input: (batch_size, context_size, embedding_size(in_channels))
        :return:
        '''
        input_ = input_.permute(0, 2, 1)
        y = []
        for layer in self.cnn_layers:
            x = layer(input_)
            x = self.pool(x).squeeze(-1)
            y.append(x)
        y = torch.cat(y, dim=1)
        return y

In [22]:
# BiLSTM 
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, bidirectional=False):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bi_dir = bidirectional
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=bidirectional)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers *(2 if self.bi_dir else 1), x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers *(2 if self.bi_dir else 1), x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        return out[:, -1, :]

In [23]:
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features)
    
    def forward(self, x, adj):
        # x: [batch_size, seq_len, in_features]
        # adj: [batch_size, seq_len, seq_len]
        x = torch.matmul(adj, x)  # Graph propagation
        x = self.linear(x)
        return F.relu(x)

class TextGCN(nn.Module):
    def __init__(self, embed_dim=128, gcn_dim=64):
        super().__init__()
        
        # GCN参数
        self.gcn1 = GCNLayer(embed_dim, gcn_dim)
        self.gcn2 = GCNLayer(gcn_dim, gcn_dim)
    
    def build_cooccurrence_matrix(self, batch):
        """构建批量共现矩阵"""
        batch_adj = []
        for seq in batch:
            seq_len = seq.size(0)
            window_size = 3
            adj = torch.zeros((seq_len, seq_len))
            
            # 滑动窗口统计共现
            for i in range(seq_len):
                start = max(0, i - window_size)
                end = min(seq_len, i + window_size + 1)
                adj[i, start:end] = 1.0
                adj[start:end, i] = 1.0
            
            batch_adj.append(adj)
        return torch.stack(batch_adj).to(seq.device)

    def forward(self, x_ori, x_emb):
        # x: [batch_size, seq_len]
        # embeddings = self.embedding(x)  # [batch, seq_len, embed_dim]
        
        # 构建共现图邻接矩阵
        adj = self.build_cooccurrence_matrix(x_ori)  # [batch, seq_len, seq_len]
        
        # GCN处理
        gcn_out = self.gcn1(x_emb, adj)
        gcn_out = self.gcn2(gcn_out, adj)  # [batch, seq_len, gcn_dim]
        
        # 池化聚合
        gcn_pooled = torch.mean(gcn_out, dim=1)  # [batch, gcn_dim]
        # text_pooled = torch.mean(embeddings, dim=1)  # [batch, embed_dim]
        
        # 特征融合
        combined = gcn_pooled # torch.cat([text_pooled, gcn_pooled], dim=1)
        return combined

In [24]:
# 多层神经网络：
class SentimentClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc = nn.Linear(input_dim, 32)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.Linear(32, 32)
        self.dropout2 = nn.Dropout(0.2)
        self.dense2 = nn.Linear(32, output_dim)
 
    def forward(self, x):
        x = F.relu(self.fc(x))
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        x = self.dropout2(x)
        x = self.dense2(x)        
        return x

# # 单层神经网络：
# class SentimentClassifier(nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super().__init__()
#         self.fc = nn.Linear(input_dim, output_dim)
 
#     def forward(self, x):
#         return self.fc(x)

In [25]:
# 定义模型
class FinalModel(nn.Module):
    def __init__(self, 
                 maxlen, vocab_size, embed_dim, num_heads, ff_dim,
                tcnn_ks = [3,5,7], tcnn_nc = [16,16,16],
                 lstm_hs = 128, lstm_nlyr = 2, lstm_bd = True,
                 gcn_gd = 64,
                ):
        super(FinalModel, self).__init__()
        ## emb层
        self.token_emb = nn.Embedding(vocab_size, embed_dim)
        ## Tsfm部分：
        self.tsfm = TransformerModel(
            maxlen, 
            vocab_size, 
            embed_dim, 
            num_heads, 
            ff_dim
        ) # embed_dim
        ## TextCNN部分：
        self.textcnn = TextCNN(
            embedding_dim = embed_dim, 
            kernel_sizes = tcnn_ks,
            num_channels = tcnn_nc
        ) # sum(tcnn_nc)
        ## BiLSTM:
        self.lstm = LSTMClassifier(
            input_size = embed_dim, 
            hidden_size = lstm_hs, 
            num_layers = lstm_nlyr,  
            bidirectional = lstm_bd
        ) # lstm_hs * (2 if lstm_bd else 1)
        ## gcn: 
        self.gcn = TextGCN(
            embed_dim, 
            gcn_gd
        )
        self.mix = nn.Sequential(
            ## 基于这个数字 embed_dim + sum(tcnn_nc) + lstm_hs * (2 if lstm_bd else 1) ，做一个全连接神经网络吧。
            nn.Linear(
                embed_dim + embed_dim + sum(tcnn_nc) + lstm_hs * (2 if lstm_bd else 1) + gcn_gd, # 
                ff_dim
            ),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(ff_dim, ff_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(ff_dim, type_of_class)
        )
        self.sc_net = SentimentClassifier(sc_input_dim, type_of_class)
        
    def forward(self, x, x_tfidf):
        x_ori = x
        x_emb = self.token_emb(x)#.to(device)
        
        ## 把emb直接平均起来，充作短路特征
        emb_fea = torch.mean(x_emb, dim=1)
        ## Tsfm部分：
        x_tsfm = self.tsfm(x_emb) 
        ## TextCNN部分：
        x_tcnn = self.textcnn(x_emb)
        ## BiLSTM部分:
        x_lstm = self.lstm(x_emb)
        ## gcn: 
        x_gcn = self.gcn(x_ori, x_emb)
        
        ## 综合：
        x_cat = torch.cat(
            [
                emb_fea, 
                x_tsfm, x_tcnn, x_lstm,
                x_gcn
            ], axis=1
        )        
        return F.log_softmax(
            self.mix(x_cat) + self.sc_net(x_tfidf) , # 
            dim=-1
        )

In [26]:
# for x in train_loader:
#     break

# fnl = FinalModel(maxlen, vocab_size, embed_dim=128, num_heads=8, ff_dim=64)

# fnl(x[0], x[1])

# 构建模型以及训练

In [27]:
model = FinalModel(maxlen, vocab_size, embed_dim=128, num_heads=8, ff_dim=64)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [28]:
model

FinalModel(
  (token_emb): Embedding(8000, 128)
  (tsfm): TransformerModel(
    (embedding_layer): TokenAndPositionEmbedding(
      (pos_emb): Embedding(800, 128)
    )
    (transformer_block): TransformerBlock(
      (att): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
      )
      (ffn): Sequential(
        (0): Linear(in_features=128, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=128, bias=True)
      )
      (layernorm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (layernorm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
    (global_avg_pool): AdaptiveAvgPool1d(output_size=1)
  )
  (textcnn): TextCNN(
    (cnn_layers): ModuleList(
      (0): Sequential(
        (0): Conv1d(128, 16, kernel_size=(3,), stride=(1,))
        (1): BatchNor

In [29]:
# 训练和评估模型
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=2):
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        for inputs, input_tfidfs, targets in tqdm.tqdm(train_loader):
            inputs, input_tfidfs, targets = inputs.to(device), input_tfidfs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs, input_tfidfs)
            loss = criterion(outputs, targets)
            loss.requires_grad_(True)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {train_loss/len(train_loader)}, Accuracy: {100.*correct/total}%')
        
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            total_predicted = []
            total_label = []
            for inputs, input_tfidfs, targets in tqdm.tqdm(val_loader):
                inputs, input_tfidfs, targets = inputs.to(device), input_tfidfs.to(device), targets.to(device)
                outputs = model(inputs, input_tfidfs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                total_predicted += list(predicted.cpu())
                total_label += list(targets.cpu())
        f1 = f1_score(total_label, total_predicted, average='macro')
                
        print(f'Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100.*correct/total}%, f1 score is {f1}')

In [30]:
## 目前先尝试只使用gcn来跑。
## 后续我们再看要不要增加其他的模型结构来跑。

In [31]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1)

100%|███████████████████████████████████████| 4375/4375 [26:39<00:00,  2.73it/s]


Epoch 1/1, Loss: 0.4351307381272316, Accuracy: 86.92928571428571%


100%|███████████████████████████████████████| 1875/1875 [05:57<00:00,  5.24it/s]


Validation Loss: 0.2343884338264664, Accuracy: 92.82%, f1 score is 0.9067184729116341


In [32]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=4) 

100%|███████████████████████████████████████| 4375/4375 [26:43<00:00,  2.73it/s]


Epoch 1/4, Loss: 0.22300845012579645, Accuracy: 93.17785714285715%


100%|███████████████████████████████████████| 1875/1875 [05:51<00:00,  5.33it/s]


Validation Loss: 0.19934430090188981, Accuracy: 93.875%, f1 score is 0.9232486830112003


100%|███████████████████████████████████████| 4375/4375 [26:51<00:00,  2.71it/s]


...

Validation Loss: 0.19579365724027156, Accuracy: 94.08333333333333%, f1 score is 0.9271034543624819


100%|███████████████████████████████████████| 4375/4375 [25:48<00:00,  2.83it/s]


Epoch 4/4, Loss: 0.11539019079501076, Accuracy: 96.28571428571429%


100%|███████████████████████████████████████| 1875/1875 [05:42<00:00,  5.47it/s]


Validation Loss: 0.20654332947935908, Accuracy: 94.26833333333333%, f1 score is 0.9283153154642002


In [33]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1) 
## 意义不大了

100%|███████████████████████████████████████| 4375/4375 [25:58<00:00,  2.81it/s]


Epoch 1/1, Loss: 0.09415317839064768, Accuracy: 96.93785714285714%


100%|███████████████████████████████████████| 1875/1875 [05:39<00:00,  5.53it/s]


Validation Loss: 0.221604420032911, Accuracy: 94.00166666666667%, f1 score is 0.9249864347017996


In [34]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)

100%|███████████████████████████████████████| 4375/4375 [25:54<00:00,  2.81it/s]


Epoch 1/5, Loss: 0.07772540336156795, Accuracy: 97.43071428571429%


100%|███████████████████████████████████████| 1875/1875 [05:39<00:00,  5.52it/s]


Validation Loss: 0.2353672108337283, Accuracy: 93.965%, f1 score is 0.9249550281501006


100%|███████████████████████████████████████| 4375/4375 [25:44<00:00,  2.83it/s]


...

Validation Loss: 0.27236917958913837, Accuracy: 93.995%, f1 score is 0.9255836095186504


100%|███████████████████████████████████████| 4375/4375 [25:58<00:00,  2.81it/s]


Epoch 5/5, Loss: 0.0425870361896132, Accuracy: 98.515%


100%|███████████████████████████████████████| 1875/1875 [05:41<00:00,  5.49it/s]


Validation Loss: 0.2818319543664169, Accuracy: 94.11%, f1 score is 0.9258190264691774


In [35]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=1)

100%|███████████████████████████████████████| 4375/4375 [25:43<00:00,  2.83it/s]


Epoch 1/1, Loss: 0.0387133414902716, Accuracy: 98.67357142857144%


100%|███████████████████████████████████████| 1875/1875 [05:38<00:00,  5.54it/s]


Validation Loss: 0.27515747745550856, Accuracy: 94.0%, f1 score is 0.9252897626872325


# 保存模型

In [36]:
save_pickle_object(model, create_trained_models_path("ori_9319-20250207_1.pkl"))

/Users/minkexiu/Downloads/GitHub/Tianchi_NLPNewsClassification/trained_models/ori_9319-20250207_1.pkl


# 预测一下试试

In [37]:
# model = load_pickle_object(create_trained_models_path("ori_9319-20250207_1.pkl"))

In [38]:
with torch.no_grad():
    total_predicted = []
    for inputs, input_tfidfs, targets in tqdm.tqdm(test_loader):
        inputs, input_tfidfs, targets = inputs.to(device), input_tfidfs.to(device), targets.to(device)
        outputs = model(inputs, input_tfidfs)
        _, predicted = torch.max(outputs, 1)
        total_predicted += list(predicted)

100%|███████████████████████████████████████| 1563/1563 [04:36<00:00,  5.65it/s]


In [39]:
oot_rst = [int(x) for x in total_predicted]
len(oot_rst)

50000

In [40]:
store_data_to_newbasepath(pd.DataFrame({"label": oot_rst}), "rst-20250207_1", fmt="csv")

df.to_csv("/Users/minkexiu/Downloads/GitHub/Tianchi_NLPNewsClassification/preprocessedData/rst-20250207_1.csv", index=False)
data saved.


'/Users/minkexiu/Downloads/GitHub/Tianchi_NLPNewsClassification/preprocessedData/rst-20250207_1.csv'